In [1]:
pip install cogflow-1.9.39b8-py3-none-any.whl

Processing ./cogflow-1.9.39b8-py3-none-any.whl
  Using cached mlflow-2.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scikit_learn-1.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached flask-2.3.3-py3-none-any.whl.metadata (3.6 kB)
  Using cached databricks_cli-0.18.0-py2.py3-none-any.whl.metadata (4.0 kB)
  Using cached docker-6.1.3-py3-none-any.whl.metadata (3.5 kB)
  Using cached importlib_metadata-5.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached packaging-22.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pyarrow-10.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl.metadata (21 kB)
  Using cached shap-0.46.0-cp311-cp311-manylinux_2_12_x86

In [2]:
!pip show cogflow

Name: cogflow
Version: 1.9.39b8
Summary: COG modules
Home-page: 
Author: Sai_kireeti
Author-email: sai.kireeti@hiro-microdatacenters.nl
License: 
Location: /opt/conda/lib/python3.11/site-packages
Requires: awscli, boto3, kfp, kserve, kubernetes, minio, mlflow, numpy, pandas, ray, s3fs, scikit-learn, setuptools, tenacity, tensorflow
Required-by: 


In [15]:
import cogflow as cf
web_downloader_op=cf.load_component(url='https://raw.githubusercontent.com/Ark-kun/pipeline_components/master/components/web/Download/component.yaml')

In [16]:
print(web_downloader_op)

<function create_function_from_parameters.<locals>.pass_locals at 0x7e379791a520>


In [17]:
from cogflow import InputPath, OutputPath
def preprocess(file_path: InputPath('CSV'),
              output_file: OutputPath('parquet')):
    import pandas as pd
    df = pd.read_csv(file_path, header=0, sep=";")
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.to_parquet(output_file)

In [18]:
preprocess_op=cf.create_component_from_func(
        func=preprocess,
        output_component_file='preprocess-component.yaml')

In [19]:
def training(file_path: InputPath('parquet'))->str:
    
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    from sklearn.linear_model import ElasticNet
    import os
    from datetime import datetime
    import numpy as np
    import cogflow as cf
    import json
     
    df = pd.read_parquet(file_path)
    
    cf.autolog()
    
    target_column='quality'
    train_x, test_x, train_y, test_y = train_test_split(df.drop(columns=[target_column]),
                                                    df[target_column], test_size=.25,
                                                    random_state=1337, stratify=df[target_column])  

    with cf.start_run() as run:
        alpha =  0.5
        l1_ratio =  0.5
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        cf.log_param("alpha",alpha)
        cf.log_param("l1_ratio",l1_ratio)
        
        
        model_name = "wine-elasticnet"
        run_id = run.info.run_id
        result = cf.log_model(lr, "model", registered_model_name= "wine-elasticnet")

        print(f"Artifact_uri", run.info.artifact_uri)
        print(f"Artifact_path", run.info.artifact_uri)
        return f"{run.info.artifact_uri}/{result.artifact_path}"
        

In [20]:
training_op=cf.create_component_from_func(
        func=training,
        output_component_file='train-component.yaml')

In [21]:
def serving(model_uri,name):
    import cogflow as cf
    cf.serve_model_v1(model_uri,name)
    

In [22]:
kserve_op=cf.create_component_from_func(func=serving,
        output_component_file='kserve-component.yaml')

In [23]:
def getmodel(name):
    import cogflow as cf
    cf.get_model_url(name)
    

In [24]:
getmodel_op=cf.create_component_from_func(func=getmodel,
        output_component_file='kserve-component.yaml')


In [25]:
@cf.pipeline(name="pipeline", description="WINE pipeline")
def wine_pipeline(url,isvc):
    # Download dataset
    web_downloader_task = web_downloader_op(url=url)
    
    # Adjust dataset for training
    preprocess_task = preprocess_op(file=web_downloader_task.outputs['data'])
    
    # Training
    train_task = (training_op(file=preprocess_task.outputs['output']));
    
    # Serving the model
    kserve_task=kserve_op(model_uri=train_task.output,name=isvc)
    #kserve_task=kserve_task.AddModelAccess()
    kserve_task.after(train_task)
    
    # Describe model
    getmodel_task=getmodel_op(isvc)
    getmodel_task.after(kserve_task)
    
    # Infer model
    #API request internally in this cluster

In [26]:
client = cf.client()
client.create_run_from_pipeline_func(
    wine_pipeline,
    arguments={
        "url": "https://raw.githubusercontent.com/Barteus/kubeflow-examples/main/e2e-wine-kfp-mlflow/winequality-red.csv",
    "isvc":"sample-final-bola-verge332"}
)


RunPipelineResult(run_id=ba82a539-f7c4-4cc7-bc23-03ec3f0c709f)

In [ ]:
# Important to delete served models for resources
import cogflow as cf
cf.delete_served_model("sample-final-bola-verge332")

In [26]:

import cogflow
res = cogflow.save_model_details_to_db(registered_model_name="FanzD7VJHc")
print(res)


POST request successful
{'status_code': 201, 'message': 'Created new model.', 'data': {'name': 'FanzD7VJHc', 'version': '1', 'type': 'sklearn', 'description': 'FanzD7VJHc model', 'id': 9, 'last_modified_time': '2024-10-11T15:00:02.125188', 'register_date': '2024-10-11T15:00:02.125181'}, 'pagination': None}


In [28]:
model_id = res["data"]["id"]
print(model_id)

9


In [33]:


artifact_uri=cogflow.get_artifact_uri(artifact_path="TD3_HalfCheetah-v2_0")
print(artifact_uri)

s3://mlflow/0/8e26c211a995496299c9b2a210103aea/artifacts/TD3_HalfCheetah-v2_0


In [32]:
cogflow.save_model_uri_to_db(model_id="9", model_uri=artifact_uri)

POST request failed with status code 422


Exception: Request failed